Import cryptocurrency data from CoinMarketCap's API

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

Import pandas and set display to see all available columns

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Normalise the data and improve readability with a dataframe

In [ ]:
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

In [ ]:
import os
from time import time
from time import sleep

Put the API call into a function that appends each call to the dataframe

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    parameters = {
        'start':'1',
        'limit':'15',
        'convert':'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

    df2 = pd.json_normalize(data['data'])
    df2['timestamp'] = pd.Timestamp('now')
    df = df.append(df2)

    #If you want to output the dataframe to a csv, uncomment the below code, and change the apove line "df = df.append(df2)" to just "df"
    
    #if not os.path.isfile(r"addfilepathhere"):
    #    df.to_csv(r"addfilepathhere", header = "column_names")
    #else:
    #    df.to_csv(r"addfilepathhere", header = "column_names", mode = "a", header = False)

Specify how many times you want to run the API function

In [ ]:
#Run data x number of times, only running once for demo purposes
for i in range(1):
    api_runner()
    print("API Runner completed successfully")
    #sleep(60)

Clean and format data

In [ ]:
df3 = pd.set_option("display.float_format", lambda x: "%.5f" % x)

In [ ]:
df3 = df.groupby("name", sort = False)[["quote.USD.percent_change_1h", "quote.USD.percent_change_24h", "quote.USD.percent_change_7d", "quote.USD.percent_change_30d", "quote.USD.percent_change_60d"]].mean()


In [ ]:
df4 = df3.stack()

In [ ]:
df5 = df4.to_frame(name="values")

In [ ]:
index = pd.Index(range(75))

In [ ]:
df6 = df5.reset_index()

Rename columns

In [ ]:
df7 = df6.rename(columns={"level_1": "percent_change"})

In [ ]:
df7["percent_change"] = df7["percent_change"].replace(["quote.USD.percent_change_1h", "quote.USD.percent_change_24h", "quote.USD.percent_change_7d", "quote.USD.percent_change_30d", "quote.USD.percent_change_60d" ], ["1hr", "24hr", "7d", "30d", "60d"])

Import and plot data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x="percent_change", y="values", hue="name", data=df7, kind="point")